In [ ]:
%matplotlib inline

# Why use Python for Data Analysis?
What benefits does using Python give us over using Excel for data analysis?

* Automated reproducible analysis
* Working with large datasets
* Analysis scripts as batch processes

# The big four (five) of data analysis

### load
Reading data from different sources (databases, csv files, spreadsheets, APIs) and loading that in to an object we can work with.
### manipulate
Data cleaning and preparation to form a dataset that we can use to drive our analysis.
### analyse
Inspection of our data to answer a particular question.
### visualise 
Reporting results in a clear way that conveys the findings of the analysis.
### automate
Reproducible analysis.

# Python packages for Data Analysis

For this introduction we will use the below Python modules. There are of course many more modules aimed at scientifc python and more advanced visualisations.

* `numpy` - fundamental package for scientific computing with Python
* `pandas` - powerful Python data analysis toolkit
* `matplotlib` - 2D plotting library

# Getting started with Numpy and Pandas
**The** library for Data Analysis in Python.

In [ ]:
import numpy as np
import pandas as pd

## Series
One-dimensional labelled array - a bit like a Python `dict`.

In [ ]:
# Create a series from 5 random numbers
s_rand = pd.Series(np.random.rand(5))
print(s_rand)

In [ ]:
# Create a series from a python dictionary
s_dict = pd.Series({'a': 34, 'b': 45, 'c': 56, 'd': 23, 'e':67})
print(s_dict)

In [ ]:
# Get items based on their label
print(s_rand[3])
print(s_dict['a'])

## Index
The labels that are assigned to our Series are an object called an **Index**. When working with *Series* and later *DataFrame*s it is important to understand the Index of your data. It is worth noting that this index doesn't have to be unique and it can contain multiple labels that create index levels.

In [ ]:
# We can set a custom index when creating a series
s_rand = pd.Series(np.random.rand(5), index=['a', 'b', 'c', 'd', 'e'])
print(s_rand)
print(s_rand.index)

## DataFrame

2-dimensional labelled array, a bit like a database table, a spreadsheet or a `dict` or `Series`. Imagine each column in the table is represented as a `Series` and our `dict` labels these with a given column name - the result is a 2-dimensional dataset with labels for the **rows**(i.e the row index) **and the columns** (i.e the column index).

In [ ]:
d = {'one': pd.Series(np.random.rand(3), index=['a', 'b', 'c']),
     'two': pd.Series(np.random.rand(3), index=['a', 'b', 'c'])}

pd.DataFrame(d)
#print(df)
#print(df.index)
#print(df.columns)

# Working with DataFrames
The `DataFrame` is a fundamental data structure for almost all data analysis using Pandas from loading data to visualisation. Let's start with the first of our big four - **loading data!**

## Loading Data
Loading data in Pandas is straightfoward. There is built-in support for loading data from flat files, Excel files, SQL tables, JSON etc...

The functions for loading data have many options than can handle different forms of data within these formats.

As a basic example, this repository includes a CSV file that contains some data about public transport journies in London.

In [ ]:
df = pd.read_csv('../data/tfl-journeys-type.csv')

# Get the first 5 rows
df.head()

### Inspecting DataFrame

In [ ]:
# Get the index of the dataframe
df.index

In [ ]:
# Get the dimensions of the dataset
df.shape

In [ ]:
# Get the column names
df.columns.values

In [ ]:
# Get the data types
df.dtypes

In [ ]:
# Some basic stats about the DataFrame
df.describe()

### Selecting Data
Often we will want to select a subset of our data. There are lots of different ways to select data from a Series or DataFrame. The most commonly used are, `loc`, `iloc` and `[]`.
#### `.loc`
*Loc*ate data using the index, i.e **selecting data by label**

In [ ]:
# Currently we just have an auto generated RangeIndex
df.loc[0]

Ok we can look up the row with the label 0, but can we re-label the rows in our DataFrame to be more useful?

In [ ]:
# Update the DataFrame to have a new index
df = df.set_index('Period and Financial year')
df

In [ ]:
df.loc['01_10/11']

In [ ]:
# we can also provide a list of labels
df.loc[['01_10/11', '02_10/11']]

#### `.iloc`
**I**nteger-**loc**ate, i.e **selecting data by position**

In [ ]:
# Get the first row using it's position - 0
df.iloc[0]

In [ ]:
df.iloc[0:5]

In [ ]:
# Both loc and iloc are multi-dimensional
df.iloc[0:5,4]

#### `[]` selection

`df[]` syntax is most often used for simple column selection. Selecting a column from a DataFrame returns a Series.

In [ ]:
df['Tram Journeys (m)'].head()

#### Boolean indexing
We can also select data based on some boolean conditions, similar to using `WHERE` in SQL.

In [ ]:
# The following creates a boolean Series we can use to filter our DataFrame
bus_filter = df['Bus journeys (m)'] > 200
bus_filter.head()

In [ ]:
df[bus_filter]

In [ ]:
df[(bus_filter) & (df['Underground journeys (m)'] > 115)]

### Excercise
This repository also includes a CSV file (`../data/Walking-Cycling.csv`) about the walking and cycling habits of residents of various London boroughs as well as averages for the country.

1. Read the CSV file in to a DataFrame
1. Inspect the DataFrame to get some basic information
1. Set the index of the DataFrame to be a compound index from the columns `Local Authority` and `Year`.
1. Using `iloc` implement an equivalent of the `head` function

In [ ]:
## <-- your code here --> ##
df = pd.read_csv('../data/Walking-Cycling.csv')
print(df.shape)
df['Frequency'].unique()
df = df.set_index(['Year', 'Local Authority'])
df = df[['Walking_%', 'Cycling_%']]
df.iloc[0:5]
cycling = df['Walking_%'] > 50
df[cycling]

## Manipulating Data

The public transport dataset we have needs some cleaning and preparation before we do our analysis. `DataFrame` objects can be manipulated to deals with missing or dirty data.

* working with missing data
* handling data types
* adding and removing columns

We can remove column by selecting a subset of columns and reassigned the `DataFrame`.

In [ ]:
df = pd.read_csv('../data/tfl-journeys-type.csv')

### Working with missing data
Some data relating to newer transport methods has missing data for the older reporting periods. This can be seen by calling `count` on our `DataFrame`.

In [ ]:
df.count()

Pandas handles these missing values pretty well and won't include these when performing any calculations such as `sum` or `mean`. However on occasion we might want to fill these missing values.

In [ ]:
df['Emirates Airline Journeys (m)'] = df['Emirates Airline Journeys (m)'].fillna(value='dfg')
df

Or we might decide to remove the labels from our `DataFrame` where we have missing values.

In [ ]:
df = df.dropna()
df.count()

### Handling data types
Often data that we read isn't imported in the right format and we have to do some work to clean up the data and get everything in the right format.

In [ ]:
df = pd.read_csv('../data/tfl-journeys-type.csv')
df.dtypes

In [ ]:
df['Period beginning'] = pd.to_datetime(df['Period beginning'])
df['Period ending'] = pd.to_datetime(df['Period ending'])
df.dtypes

### Adding and removing columns

In [ ]:
df = df[['Period and Financial year', 'Reporting Period', 'Bus journeys (m)', 'Underground journeys (m)']]
df.head()

We can insert columns in a similar way to how we would add a new key to a Python `dict`. Here we are using the apply function to create a new column called 'Financial Year'.

In [ ]:
df['Financial Year'] = df['Period and Financial year'].apply(lambda x:x[3:])
df.head()

Let's use this new column to create a sensible index for our DataFrame

In [ ]:
df = df.set_index(['Financial Year', 'Reporting Period'])
df.head()

In [ ]:
df = df[['Bus journeys (m)', 'Underground journeys (m)']]
df.head()

In [ ]:
df.loc[('14/15', 1)]

## Basic Analysis
Once we have prepared our data, we can now begin the real analysis. Typically we are looking to answer some questions about the dataset we are working with. The answers to these questions will hopefully give us the information we need to make informed decisions based on this data.

This repository also includes a CSV file called 'weather_data.csv'. This file contains some data about the weather in 3 different European cities. 

We can analyse this data to answer questions such as, 

* What is the maximum windspeed recorded in Rome? <br>
* Which city has the highest average temperature?


In [ ]:
# Read the weather data CSV
df = pd.read_csv('../data/weather_data.csv')
df

To compare different cities we need to group this dataset by the `city` column. You can visualise this grouping here
![group](../img/group.png)

In [ ]:
# Group by the city
g = df.groupby(['city'])
g

This gives us a DataFrameGroupBy object which we can then inspect to answer the questions we had about this data. The grouped object provides a mapping between group name and the group contents.

In [ ]:
# Get the groups in our group-by
print(g.groups)

# Print the contents if each group
for name, group in g:
    print(name)
    print(group)

In [ ]:
# Get the max windspeed in Rome
print(g.get_group('Rome')['windspeed'].max())

In [ ]:
# Get the city with maximum average windspeed
g.mean()['windspeed'].max()

### Exercise
Group our weather dataset by `date` to give 5 groups (one for each day). Use this grouping to find the day which was the coldest. (i.e minimum average temperate across the 3 cities)

In [ ]:
## <-- your code here --> ##
g = df.groupby(['date'])
g
# Get the groups in our group-by
print(g.groups)

# Print the contents if each group
for name, group in g:
    print(name)
    print(group)
    
g.mean()['temperature'].idxmin()

## Visualisation
Visualising data is important because often we want to present the findings of the analysis in a way that is easy to interpret. Data visualisation is itself a broad topic that explores how best to communicate information using various visualisation techniques.

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

### Built-in Visualisation

Many Pandas objects have some built-in plotting and visualisation functions to create some basic plots.

In [ ]:
# Plot a series of 5 random numbers
s_rand = pd.Series(np.random.rand(5))
s_rand.plot()

In [ ]:
# Plot a histogram of a random normal distribution
s_normal_rand = pd.Series(np.random.normal(size=1000000))
s_normal_rand.hist(buckets=20)

In [ ]:
g = df.groupby(['city'])
g.plot()

### Customising Plots with Matplotlib

# Advanced Exercise

Return to the Walking-Cycling dataset. Extract the data relating to the 8 regions of England. Create a matplotlib figure that contains 4 subplots (one for each Walking-Cycling frequency). Plot a multi-line graph in each subplot that shows how the excerise habits of the people in these regions has changed over the period.

Create a single legend for the figure.